# Konfigrasyon

In [ ]:
FILENAME = "siyasi.mp4"

# Altyazı Parametreleri
font = "Helvetica"
color='white'
highlight_color='yellow'
stroke_color='black'
stroke_width=1.5


# Gerekli Kütüphane ve Program Kurulumları

In [ ]:
!pip install --quiet srt_file_translator

!pip install --quiet ipython-autotime
%load_ext autotime

!pip install --quiet moviepy==2.0.0.dev2
!pip install --quiet imageio==2.25.1
!pip install --quiet ffmpeg-python==0.2.0
!pip install --quiet faster-whisper==0.7.0
!pip install --quiet python-docx

In [ ]:
!apt install  imagemagick
!cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml

# Core Functions


In [ ]:
from faster_whisper import WhisperModel
import ffmpeg
import json
from docx import Document
import re
from moviepy.editor import TextClip, CompositeVideoClip, concatenate_videoclips,VideoFileClip, ColorClip
import numpy as np

In [ ]:
#
def extract_sound_file(fileName):
    # MP4 dosya adını MP3 formatına dönüştür
    audiofilename = fileName.replace(".mp4", '.mp3')

    # ffmpeg ile dosyayı okumak için giriş akışı oluştur
    input_stream = ffmpeg.input(fileName)

    # Giriş akışından sadece ses verisini al
    audio = input_stream.audio

    # Çıktı olarak kullanılacak ses dosyasını ve adını ayarla
    output_stream = ffmpeg.output(audio, audiofilename)

    # Eğer hedef dosya zaten varsa üzerine yaz
    output_stream = ffmpeg.overwrite_output(output_stream)

    # ffmpeg işlemini başlat ve ses dosyasını oluştur
    ffmpeg.run(output_stream)

    # Oluşturulan ses dosyasının adını döndür
    return audiofilename

#
# Belirli bir model boyutu için WhisperModel yükler.
# `model_size` parametresi modelin boyutunu belirler (varsayılan olarak "medium").
def load_model(model_size="medium"):
    # İstenen boyutta bir Whisper modeli oluşturur ve döndürür.
    return WhisperModel(model_size)

#
# Verilen model ve ses dosyası adı kullanılarak ses dosyasından segmentler oluşturur.
# `model` parametresi, transkripsiyon yapmak için kullanılacak modeli,
# `audiofilename` parametresi ise transkribe edilecek ses dosyasının adını belirtir.
def create_segments(model, audiofilename):
    # Modelin `transcribe` metodunu çağırarak ses dosyasını transkribe eder.
    # `word_timestamps=True` argümanı, her kelimenin başlangıç ve bitiş zaman damgalarını döndürür.
    segments, info = model.transcribe(audiofilename, word_timestamps=True)
    # Oluşturulan segmentleri döndürür.
    return segments

#
# Transkribe edilen segmentleri yazdırır.
# `segments` parametresi, transkribe edilen kelimelerin segmentlerini içerir.
def print_segments(segments):
    # Segmentleri liste olarak işler.
    segments = list(segments)
    # Her bir segment ve içerdiği kelimeler üzerinde döngü yapar.
    for segment in segments:
        for word in segment.words:
            # Her kelimenin başlangıç ve bitiş zamanlarını ve kelimenin kendisini yazdırır.
            print("[%.2fs -> %.2fs] %s" % (word.start, word.end, word.word))

#
# Transkribe edilmiş segmentlerden kelime düzeyinde bilgileri işler.
# `segments` parametresi, transkribe edilen kelimelerin segmentlerini içerir.
def process(segments):
    segments = list(segments)  # Segmentleri liste olarak işler.

    wordlevel_info = []
    # Her bir segment ve içerdiği kelimeler üzerinde döngü yapar.
    for segment in segments:
        for word in segment.words:
            # Her kelimenin metnini, başlangıç ve bitiş zamanlarını içeren bir sözlük oluşturur.
            wordlevel_info.append({'word': word.word, 'start': word.start, 'end': word.end})

    # İşlenmiş kelime düzeyinde bilgileri döndürür.
    return wordlevel_info

#
# Verilen veriyi JSON formatında bir dosyaya kaydeder.
# `data` parametresi, JSON'a dönüştürülecek veriyi içerir.
def dump_json(data):
    # 'data.json' adında bir dosya oluşturur ve yazma modunda açar.
    with open('data.json', 'w') as f:
        # Veriyi JSON formatında dosyaya yazar. İndentasyon ile okunabilirliği artırır.
        json.dump(data, f, indent=4)

#
# Bir JSON dosyasını okur ve içeriğini döndürür.
# `fileName` parametresi, okunacak JSON dosyasının adını belirtir (varsayılan olarak 'data.json').
def read_json(fileName='data.json'):
    # Belirtilen dosyayı okuma modunda açar.
    with open(fileName, 'r') as f:
        # Dosya içeriğini JSON olarak yükler ve bir değişkene atar.
        wordlevel_info_modified = json.load(f)

    # Yüklenen veriyi döndürür.
    return wordlevel_info_modified

#
# Alt yazıyı satrı satır forma çevirmek için
def split_text_into_lines(data):
    MaxChars = 30  # Bir satırdaki maksimum karakter sayısı.
    MaxDuration = 2.5  # Bir satırın maksimum süresi (saniye cinsinden).
    MaxGap = 1.5  # İki kelime arasındaki maksimum boşluk süresi (saniye cinsinden).

    subtitles = []  # Sonuç alt yazı satırlarını saklamak için liste.
    line = []  # Geçici olarak kelimeleri saklamak için kullanılan liste.
    line_duration = 0  # Geçici satırın toplam süresi.
    line_chars = 0  # Açıklama gereksiz, kullanılmıyor.

    for idx, word_data in enumerate(data):  # Girdi verisindeki her kelime için döngü.
        word = word_data["word"]
        start = word_data["start"]
        end = word_data["end"]

        line.append(word_data)  # Geçici satıra kelime eklenir.
        line_duration += end - start  # Geçici satırın süresi güncellenir.

        temp = " ".join(item["word"] for item in line)  # Geçici satırdaki kelimeler birleştirilir.

        # Yeni bir kelime eklenmesiyle karakter sayısı veya süre sınırını aşılıp aşılmadığının kontrolü.
        new_line_chars = len(temp)
        duration_exceeded = line_duration > MaxDuration
        chars_exceeded = new_line_chars > MaxChars

        # İki kelime arasındaki boşluk süresinin kontrolü.
        if idx > 0:
            gap = word_data['start'] - data[idx-1]['end']
            maxgap_exceeded = gap > MaxGap
        else:
            maxgap_exceeded = False  # İlk kelime için boşluk süresi kontrolü yapılmaz.

        # Karakter sayısı, süre veya boşluk süresi sınırı aşıldıysa, geçici satır sonlandırılır.
        if duration_exceeded or chars_exceeded or maxgap_exceeded:
            if line:  # Eğer geçici satır boş değilse, sonuç listesine eklenir.
                subtitle_line = {
                    "word": " ".join(item["word"] for item in line),
                    "start": line[0]["start"],
                    "end": line[-1]["end"],
                    "textcontents": line
                }
                subtitles.append(subtitle_line)
                line = []  # Geçici satır sıfırlanır.
                line_duration = 0

    # Döngü bittiğinde, kalan son satır (varsa) sonuç listesine eklenir.
    if line:
        subtitle_line = {
            "word": " ".join(item["word"] for item in line),
            "start": line[0]["start"],
            "end": line[-1]["end"],
            "textcontents": line
        }
        subtitles.append(subtitle_line)

    # Altyazıları döndür
    return subtitles

#
# Bu fonksiyonu, metin tabanlı JSON verisini kullanarak, bir video içerisinde altyazılar oluşturmak üzere metin klip nesneleri dizisi üretir.
def create_caption(textJSON, framesize,font=font,color=color, highlight_color=highlight_color,stroke_color=stroke_color,stroke_width=stroke_width):
    wordcount = len(textJSON['textcontents'])  # JSON'da bulunan kelime sayısı.
    full_duration = textJSON['end'] - textJSON['start']  # Altyazının toplam süresi.

    word_clips = []  # Oluşturulan metin kliplerini saklamak için liste.
    xy_textclips_positions = []  # Metin kliplerinin pozisyon bilgilerini saklamak için liste.

    # Başlangıç pozisyonları ve satır genişliği.
    x_pos = 0
    y_pos = 0
    line_width = 0
    frame_width, frame_height = framesize

    x_buffer = frame_width * 0.1  # Çerçevenin kenarlarından boşluk.
    max_line_width = frame_width - 2 * x_buffer  # Maksimum satır genişliği.

    fontsize = int(frame_height * 0.075)  # Font boyutu, video yüksekliğinin %7.5'i olarak ayarlanır.

    for index, wordJSON in enumerate(textJSON['textcontents']):
        duration = wordJSON['end'] - wordJSON['start']  # Kelimenin süresi.
        # Her kelime için bir metin klipi oluşturulur.
        word_clip = TextClip(wordJSON['word'], font=font, fontsize=fontsize, color=color, stroke_color=stroke_color, stroke_width=stroke_width).set_start(textJSON['start']).set_duration(full_duration)
        word_clip_space = TextClip(" ", font=font, fontsize=fontsize, color=color).set_start(textJSON['start']).set_duration(full_duration)
        word_width, word_height = word_clip.size  # Metin klipinin boyutları.
        space_width, space_height = word_clip_space.size  # Boşluk klipinin boyutları.

        # Yeni kelimenin mevcut satıra sığdırılması veya yeni bir satıra geçilmesi.
        if line_width + word_width + space_width <= max_line_width:
            xy_textclips_positions.append({
                "x_pos": x_pos,
                "y_pos": y_pos,
                "width": word_width,
                "height": word_height,
                "word": wordJSON['word'],
                "start": wordJSON['start'],
                "end": wordJSON['end'],
                "duration": duration
            })
            word_clip = word_clip.set_position((x_pos, y_pos))
            x_pos += word_width + space_width  # X pozisyonunu güncelle.
            line_width += word_width + space_width  # Satır genişliğini güncelle.
        else:
            # Yeni satıra geçiş yapılır.
            x_pos = 0
            y_pos += word_height + 10  # Y pozisyonunu güncelle.
            line_width = word_width + space_width  # Satır genişliğini sıfırla.

            xy_textclips_positions.append({
                "x_pos": x_pos,
                "y_pos": y_pos,
                "width": word_width,
                "height": word_height,
                "word": wordJSON['word'],
                "start": wordJSON['start'],
                "end": wordJSON['end'],
                "duration": duration
            })
            word_clip = word_clip.set_position((x_pos, y_pos))
            x_pos += word_width + space_width  # X pozisyonunu güncelle.

        word_clips.append(word_clip)
        word_clips.append(word_clip_space)  # Boşluk klipi de eklenir.

    # Vurgulanan kelimeler için ayrı metin klipleri oluşturulur.
    for highlight_word in xy_textclips_positions:
        word_clip_highlight = TextClip(highlight_word['word'], font=font, fontsize=fontsize, color=highlight_color, stroke_color=stroke_color, stroke_width=stroke_width).set_start(highlight_word['start']).set_duration(highlight_word['duration'])
        word_clip_highlight = word_clip_highlight.set_position((highlight_word['x_pos'], highlight_word['y_pos']))
        word_clips.append(word_clip_highlight)  # Vurgulanan kelime klipleri listeye eklenir.

    return word_clips, xy_textclips_positions

# Dönüşüm Fonksiyonları

In [ ]:
#
def segments_to_srt(segments, output_filename):
    # Dosyayı yazma modunda aç ve UTF-8 olarak kodla
    with open(output_filename, 'w', encoding='utf-8') as file:
        # Segment listesindeki her bir segment için döngü başlat
        for i, segment in enumerate(segments, start=1):
            # Segmentin başlangıç ve bitiş zamanını al
            start = segment["start"]
            end = segment["end"]
            # Zamanları SRT formatına uygun şekilde formatla
            start_srt = "%02d:%02d:%02d,%03d" % (int(start / 3600), int(start / 60 % 60), int(start % 60), int(start * 1000 % 1000))
            end_srt = "%02d:%02d:%02d,%03d" % (int(end / 3600), int(end / 60 % 60), int(end % 60), int(end * 1000 % 1000))
            # Sıra numarası, formatlanmış zaman aralığı ve segment metnini dosyaya yaz
            file.write(f"{i}\n")
            file.write(f"{start_srt} --> {end_srt}\n")
            file.write(f"{segment['word']}\n\n")  # Segment metni

#
def srt_to_docx(srt_file_path="transcription.srt", docx_file_path="transcription.docx"):
    # Yeni bir Word belgesi oluştur
    doc = Document()

    # SRT dosyasından okunan tüm metinleri birleştirmek için boş bir string
    full_text = ''

    # SRT dosyasını oku ve içeriğini bir değişkene ata
    with open(srt_file_path, 'r', encoding='utf-8') as file:
        srt_content = file.read()

    # SRT içeriğini alt yazı girişlerine ayır
    subtitles = re.split(r'\n\n+', srt_content)

    # Her alt yazı girişini işle ve tam metni oluştur
    for subtitle in subtitles:
        # Zaman damgalarını atla ve sadece metin kısmını al
        lines = subtitle.split('\n')[2:]
        # Metin satırlarını birleştir
        subtitle_text = ' '.join(lines)
        # Tam metni güncelle
        full_text += subtitle_text + ' '

    # Tam metni Word belgesine ekle
    doc.add_paragraph(full_text)

    # Word belgesini kaydet
    doc.save(docx_file_path)

#
def seconds_to_srt_time(seconds):
    """Saniye cinsinden zamanı saat:dakika:saniye,milisaniye formatına çevir."""
    # Saniyeyi saat, dakika, saniye ve milisaniyeye çevir
    ms = int((seconds - int(seconds)) * 1000)
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    s = int(seconds % 60)
    # Formatlanmış zamanı döndür
    return f"{h:02}:{m:02}:{s:02},{ms:03}"

def format_to_srt(data):
    """JSON verisini SRT formatına dönüştür."""
    srt_content = ""
    for index, item in enumerate(data, start=1):
        # Başlangıç ve bitiş zamanlarını SRT zaman formatına çevir
        start_time = seconds_to_srt_time(item['start'])
        end_time = seconds_to_srt_time(item['end'])
        # Kelimeyi al ve boşlukları temizle
        word = item['word'].strip()
        # SRT içeriğini oluştur
        srt_content += f"{index}\n{start_time} --> {end_time}\n{word}\n\n"
    return srt_content

def json_to_srt(json_file="data.json", output_filename="transcription.srt"):
    """JSON dosyasını oku ve içeriği SRT dosyasına yaz."""
    # JSON dosyasını oku
    with open(json_file, 'r') as file:
        data = json.load(file)

    # Veriyi SRT formatına dönüştür
    srt_content = format_to_srt(data)

    # Dönüştürülen içeriği SRT dosyasına yaz
    with open(output_filename, 'w', encoding='utf-8') as srt_file:
        srt_file.write(srt_content)

# Runtime

In [ ]:
soundFile = extract_sound_file(FILENAME)  # ses dosyasını oluştur

In [ ]:
model = load_model() # modeli içe aktarıyoruz

In [ ]:
segments = create_segments(model, soundFile) # # Modelden ses üzerinde gelen segmentleri alıyoruz

In [ ]:
wli = process(segments) # segmentleri işliyoruz, bu sırasda orjinal dilde .srt dosyası oluşturyuyoruz

In [ ]:
dump_json(wli) # json dosyasına kayıt etme

In [ ]:
data = read_json() # json dosyasında okuma

In [ ]:
subtitles = split_text_into_lines(data) # alt yazıları işle

In [ ]:
for line in subtitles:
  json_str = json.dumps(line, indent=4)
  print(json_str)

In [ ]:
# Giriş video dosyasını yükle
input_video = VideoFileClip(FILENAME)

# Giriş video çerçevesinin boyutunu al
frame_size = input_video.size

# Tüm alt başlık seviyesi bölümlerini saklamak için boş bir liste oluştur
all_linelevel_splits = []

# Her alt başlık için işlem yap
for line in subtitles:
    # Alt başlığı video çerçevesine eklemek için gerekli işlemleri yap
    out_clips, positions = create_caption(line, frame_size)

    # Alt başlıkların konumlarını ve boyutlarını hesapla
    max_width = 0
    max_height = 0
    for position in positions:
        x_pos, y_pos = position['x_pos'], position['y_pos']
        width, height = position['width'], position['height']
        max_width = max(max_width, x_pos + width)
        max_height = max(max_height, y_pos + height)

    # Arka plan rengini ve opaklığını ayarla
    color_clip = ColorClip(size=(int(max_width * 1.1), int(max_height * 1.1)), color=(64, 64, 64))
    color_clip = color_clip.set_opacity(.6)
    color_clip = color_clip.set_start(line['start']).set_duration(line['end'] - line['start'])

    # Alt başlığı ve arka planı birleştir
    clip_to_overlay = CompositeVideoClip([color_clip] + out_clips)
    clip_to_overlay = clip_to_overlay.set_position("bottom")

    # Oluşturulan bölümü listeye ekle
    all_linelevel_splits.append(clip_to_overlay)

# Giriş video süresini al
input_video_duration = input_video.duration

# Nihai videonun oluşturulması
final_video = CompositeVideoClip([input_video] + all_linelevel_splits)

# Nihai videonun sesini giriş videonun sesiyle aynı yap
final_video = final_video.set_audio(input_video.audio)

# Sonuç videoyu belirtilen adla kaydet
final_video.write_videofile("output.mp4", fps=24, codec="libx264", audio_codec="aac")

# Extra Features

In [ ]:
json_to_srt() # .json dosyasında .srt çıktısı almak için

# SRT to Docs

In [ ]:
srt_to_docx() # .srt dosyasında .dockx çıktısı almak için